In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import quandl
import yahoo_fin.stock_info as si

import cvxportfolio as cp

# Universe (Don't Run This Stuff)
[Under Construction, skip to next section] We define the universe of tradeable assets for HFAC. HFAC only trades micro-cap and small-cap equities (<$5 billion market cap).

## To Do
Scrape the ticker symbols of the small cap stocks using this website: https://finviz.com/screener.ashx?v=111&f=cap_small,geo_usa.

In [12]:
nasdaq_list = pd.read_csv('data/nasdaqlisted.txt', sep='|', skipfooter=1, engine='python')
other_list = pd.read_csv('data/otherlisted.txt', sep='|', skipfooter=1, engine='python')

In [19]:
symbols = pd.concat([nasdaq_list['Symbol'], other_list['NASDAQ Symbol']])

In [20]:
symbols

0        AACG
1        AACQ
2       AACQU
3       AACQW
4         AAL
        ...  
5505      ZTS
5506      ZUO
5507      ZVV
5508    ZXIET
5509     ZYME
Length: 9338, dtype: object

In [3]:
ticker_data = {}
for ticker in si.tickers_dow():
  ticker_data[ticker] = si.get_quote_table(ticker)
ticker_data

{'AAPL': {'1y Target Est': 124.81,
  '52 Week Range': '53.15 - 137.98',
  'Ask': '117.09 x 1100',
  'Avg. Volume': 153793993.0,
  'Beta (5Y Monthly)': 1.35,
  'Bid': '117.02 x 900',
  "Day's Range": '117.29 - 118.77',
  'EPS (TTM)': 3.28,
  'Earnings Date': 'Jan 26, 2021 - Feb 01, 2021',
  'Ex-Dividend Date': 'Nov 06, 2020',
  'Forward Dividend & Yield': '0.82 (0.70%)',
  'Market Cap': '1.995T',
  'Open': 118.64,
  'PE Ratio (TTM)': 35.77,
  'Previous Close': 118.64,
  'Quote Price': 117.33999633789062,
  'Volume': 73604287.0},
 'AMGN': {'1y Target Est': 252.67,
  '52 Week Range': '177.05 - 264.97',
  'Ask': '223.37 x 1100',
  'Avg. Volume': 2875052.0,
  'Beta (5Y Monthly)': 0.81,
  'Bid': '221.51 x 1000',
  "Day's Range": '221.48 - 225.09',
  'EPS (TTM)': 12.4,
  'Earnings Date': 'Jan 28, 2021 - Feb 01, 2021',
  'Ex-Dividend Date': 'Nov 13, 2020',
  'Forward Dividend & Yield': '6.40 (2.87%)',
  'Market Cap': '129.923B',
  'Open': 223.22,
  'PE Ratio (TTM)': 18.0,
  'Previous Close': 2

In [6]:
for ticker, ticker_info in ticker_data.items():
  print(f'{ticker} Market Cap: {ticker_info["Market Cap"]}')

AAPL Market Cap: 1.995T
AMGN Market Cap: 129.923B
AXP Market Cap: 90.65B
BA Market Cap: 112.691B
CAT Market Cap: 93.565B
CRM Market Cap: 234.816B
CSCO Market Cap: 173.119B
CVX Market Cap: 165.147B
DIS Market Cap: 254.922B
DOW Market Cap: 40.596B
GS Market Cap: 76.848B
HD Market Cap: 303.733B
HON Market Cap: 141.741B
IBM Market Cap: 104.2B
INTC Market Cap: 186.008B
JNJ Market Cap: 385.299B
JPM Market Cap: 349.232B
KO Market Cap: 226.346B
MCD Market Cap: 159.521B
MMM Market Cap: 99.727B
MRK Market Cap: 203.541B
MSFT Market Cap: 1.591T
NKE Market Cap: 208.756B
PG Market Cap: 345.41B
TRV Market Cap: 34.083B
UNH Market Cap: 317.57B
V Market Cap: 446.823B
VZ Market Cap: 248.451B
WBA Market Cap: 32.521B
WMT Market Cap: 425.743B


# Run this stuff

In [18]:
tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE']
start_date='2012-01-01'
end_date='2020-11-20'
returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date
                                                )['Adj. Close'].pct_change())
                for ticker in tickers]))
returns[["USDOLLAR"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,TSLA,NKE,USDOLLAR
Date,,,,,
2018-03-21,-0.002931,-0.001643,0.019256,-0.006737,0.000068
2018-03-22,-0.023871,-0.037340,-0.023473,-0.029088,0.000068
2018-03-23,-0.031436,-0.025258,-0.024458,0.003260,0.000068
2018-03-26,0.040319,0.026828,0.008755,0.019650,0.000070
2018-03-27,-0.037799,-0.044731,-0.082188,0.004097,0.000070


We compute rolling estimates of the first and second moments of the returns using a window of 250 days. We shift them by one unit (so at every day we present the optimizer with only past data).

In [ ]:
r_hat = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
Sigma_hat = returns.rolling(window=250, min_periods=250, closed='neither').cov().dropna()

r_hat.tail()

# Transaction Costs and Holding Costs

Here we define the transaction cost and holding cost model (sections 2.3 and 2.4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). The data can be expressed 
as 
- a scalar (like we're doing here), the same value for all assets and all time periods;
- a Pandas Series indexed by the asset names, for asset-specific values; 
- a Pandas DataFrame indexed by timestamps with asset names as columns, for values that vary by asset and in time.

In [ ]:
tcost_model=cp.TcostModel(half_spread=10E-4) # need: sigma, volumes (source from api)
hcost_model=cp.HcostModel(borrow_costs=1E-4) # need: dividends (source from api)

# Trading and Holding Constraints

In [ ]:
long_only = cp.LongOnly()

# Risk Model

In [ ]:
risk_model = cp.FullSigma(Sigma_hat.reset_index(level=1, drop=True))

# Full Optimization Policy

We define the single period optimization policy (section 4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). 

In [ ]:
gamma_risk, gamma_trade, gamma_hold = 5., 1., 1.

spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[gamma_risk*risk_model, gamma_trade*tcost_model, gamma_hold*hcost_model],
                                constraints=[leverage_limit])

We run a backtest, which returns a result object. By calling its summary method we get some basic statistics.

In [ ]:
market_sim=cp.MarketSimulator(returns, [tcost_model, hcost_model], cash_key='USDOLLAR') 
init_portfolio = pd.Series(index=returns.columns, data=250000.)
init_portfolio.USDOLLAR = 0
results = market_sim.run_multiple_backtest(init_portfolio,
                               start_time='2013-01-03',  end_time='2016-12-31',  
                               policies=[spo_policy, cp.Hold()], parallel=False)
results[0].summary()

The total value of the portfolio in time.

In [ ]:
results[0].v.plot(figsize=(12,5))
results[1].v.plot(figsize=(12,5))

The weights vector of the portfolio in time.

In [ ]:
results[0].w.plot(figsize=(12,6))